In [65]:
neededPackages <- c("tidyverse"
                   , "magrittr"
                   , "rstudioapi"
                   , "rlang"
                   , "ggplot2"
                    ,"reshape2"
                    ,"car"
                    ,"comprehenr"
                   #add more packages if needed here
)

for (i in neededPackages){
  if( !is.element(i, .packages(all.available = TRUE)) ) {
    install.packages(i)
  }
  library(i,character.only = TRUE)
}
rm(i, neededPackages)

wd <- getwd()
toyota.df <- read_csv("cleanedToyotadata.csv")
head(toyota.df)

Loading required package: carData

Attaching package: 'car'

The following object is masked from 'package:dplyr':

    recode

The following object is masked from 'package:purrr':

    some

Parsed with column specification:
cols(
  Price = col_double(),
  Age = col_double(),
  Mfg_Year = col_double(),
  KM = col_double(),
  Fuel_Type = col_character(),
  HP = col_double(),
  Metallic = col_double(),
  Automatic = col_double(),
  CC = col_double(),
  Doors = col_double(),
  Gears = col_double(),
  QuartTax = col_double(),
  Weight = col_double(),
  Guarantee = col_double(),
  BOVAG = col_double(),
  Period = col_double(),
  KM.sqrt = col_double(),
  Doors_clean = col_logical(),
  Fuel_clean = col_character()
)
Warning message:
"1 parsing failure.
row col   expected     actual                    file
  1  -- 19 columns 18 columns 'cleanedToyotadata.csv'
"

Price,Age,Mfg_Year,KM,Fuel_Type,HP,Metallic,Automatic,CC,Doors,Gears,QuartTax,Weight,Guarantee,BOVAG,Period,KM.sqrt,Doors_clean,Fuel_clean
13500,23,2002,46986,Diesel,90,1,0,2000,3,5,210,1165,0,1,3,216.7625,NA,NA


In [45]:
toyota.df$Doors_clean<- to_list(for (x in toyota.df$Doors) if (x>=4) 1 else 0)
toyota.df$Fuel_clean<- to_list(for (x in toyota.df$Fuel_Type) if (x=='Diesel') 1 else 0)
toyota.df <- as.data.frame(lapply(toyota.df, unlist))


In [54]:
toyota.df$Price_log<- log(toyota.df$Price)
toyota.df$KM_log<- log(toyota.df$KM)    
toyota.df$Age_log<- log(toyota.df$Age)    

In [47]:
head(toyota.df)

Price,Age,KM,Fuel_Type,HP,Metallic,Automatic,CC,Doors,Cylinders,...,QuartTax,Weight,Guarantee,BOVAG,Period,KM.sqrt,Doors_clean,Fuel_clean,Price_log,KM_log
13500,23,46986,Diesel,90,1,0,2000,3,4,...,210,1165,0,1,3,216.7625,0,1,9.510445,10.75760
13750,23,72937,Diesel,90,1,0,2000,3,4,...,210,1165,0,1,3,270.0685,0,1,9.528794,11.19735
13950,24,41711,Diesel,90,1,0,2000,3,4,...,210,1165,1,1,3,204.2327,0,1,9.543235,10.63852
14950,26,48000,Diesel,90,0,0,2000,3,4,...,210,1165,1,1,3,219.0890,0,1,9.612467,10.77896
13750,30,38500,Diesel,90,0,0,2000,3,4,...,210,1170,1,1,3,196.2142,0,1,9.528794,10.55841
12950,32,61000,Diesel,90,0,0,2000,3,4,...,210,1170,0,1,3,246.9818,0,1,9.468851,11.01863


In [59]:
mod.df = toyota.df[ , -which(names(toyota.df) %in% c("Age_log","KM.sqrt","KM","Period","Price","Fuel_Type","Doors"))]

In [60]:
fit = lm(mod.df$Price_log~.,data=mod.df)
summary(fit)


Call:
lm(formula = mod.df$Price_log ~ ., data = mod.df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.68048 -0.06611  0.00017  0.07663  0.52620 

Coefficients: (1 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  8.576e+00  2.045e-01  41.932  < 2e-16 ***
Age         -1.272e-02  4.189e-04 -30.355  < 2e-16 ***
HP           3.074e-03  4.265e-04   7.207 1.52e-12 ***
Metallic     1.118e-02  1.034e-02   1.081 0.280133    
Automatic    2.292e-02  2.431e-02   0.943 0.346090    
CC           8.341e-06  8.587e-06   0.971 0.331730    
Cylinders           NA         NA      NA       NA    
Gears       -1.062e-02  3.159e-02  -0.336 0.736771    
QuartTax     7.291e-04  1.852e-04   3.938 9.07e-05 ***
Weight       9.812e-04  1.438e-04   6.824 1.96e-11 ***
Guarantee    3.842e-02  9.916e-03   3.874 0.000117 ***
BOVAG        5.439e-02  1.707e-02   3.186 0.001511 ** 
Doors_clean -5.534e-03  1.017e-02  -0.544 0.586527    
Fuel_clean  -

In [61]:
backAIC = step(fit,direction="backward",data=mod.fit)

Start:  AIC=-2916.3
mod.df$Price_log ~ Age + HP + Metallic + Automatic + CC + Cylinders + 
    Gears + QuartTax + Weight + Guarantee + BOVAG + Doors_clean + 
    Fuel_clean + KM_log


Step:  AIC=-2916.3
mod.df$Price_log ~ Age + HP + Metallic + Automatic + CC + Gears + 
    QuartTax + Weight + Guarantee + BOVAG + Doors_clean + Fuel_clean + 
    KM_log

              Df Sum of Sq     RSS     AIC
- Gears        1    0.0017  9.9755 -2918.2
- Doors_clean  1    0.0043  9.9781 -2918.0
- Automatic    1    0.0130  9.9868 -2917.4
- CC           1    0.0138  9.9876 -2917.3
- Metallic     1    0.0171  9.9909 -2917.1
<none>                      9.9738 -2916.3
- KM_log       1    0.0587 10.0325 -2914.2
- BOVAG        1    0.1488 10.1226 -2908.0
- Guarantee    1    0.2202 10.1940 -2903.2
- QuartTax     1    0.2274 10.2012 -2902.7
- Fuel_clean   1    0.4478 10.4216 -2887.8
- Weight       1    0.6830 10.6568 -2872.3
- HP           1    0.7619 10.7357 -2867.2
- Age          1   13.5151 23.4888 -2323.8



In [63]:
summary(backAIC)


Call:
lm(formula = mod.df$Price_log ~ Age + HP + QuartTax + Weight + 
    Guarantee + BOVAG + Fuel_clean + KM_log, data = mod.df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.69275 -0.06516  0.00038  0.07668  0.53311 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  8.5293853  0.1450006  58.823  < 2e-16 ***
Age         -0.0127703  0.0004129 -30.926  < 2e-16 ***
HP           0.0030530  0.0003933   7.762 3.04e-14 ***
QuartTax     0.0007255  0.0001844   3.935 9.17e-05 ***
Weight       0.0009961  0.0001369   7.278 9.29e-13 ***
Guarantee    0.0395185  0.0098073   4.029 6.22e-05 ***
BOVAG        0.0525787  0.0168580   3.119  0.00189 ** 
Fuel_clean  -0.1598118  0.0282581  -5.655 2.28e-08 ***
KM_log      -0.0083339  0.0042337  -1.968  0.04942 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.121 on 685 degrees of freedom
Multiple R-squared:  0.8344,	Adjusted R-squared:  0.8325 
F-statistic: 431.

In [66]:
vif(backAIC)

Age         HP   QuartTax     Weight  Guarantee      BOVAG Fuel_clean 
  2.006896   2.103928   3.850625   3.422137   1.137587   1.206090   5.122301 
    KM_log 
  1.602826

In [81]:
mod.fit.int1 = lm(Price_log~Age + HP + QuartTax+(QuartTax*Fuel_clean) + Weight + Guarantee + 
    BOVAG +Fuel_clean + KM_log + (KM_log*Age),data=mod.df)

In [82]:
summary(mod.fit.int1)


Call:
lm(formula = Price_log ~ Age + HP + QuartTax + (QuartTax * Fuel_clean) + 
    Weight + Guarantee + BOVAG + Fuel_clean + KM_log + (KM_log * 
    Age), data = mod.df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.55132 -0.06752  0.00282  0.06650  0.48150 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)          8.3740562  0.1395902  59.990  < 2e-16 ***
Age                 -0.0044056  0.0019673  -2.239  0.02545 *  
HP                   0.0028344  0.0003708   7.644 7.17e-14 ***
QuartTax            -0.0004822  0.0002243  -2.150  0.03189 *  
Fuel_clean          -0.5405066  0.0504380 -10.716  < 2e-16 ***
Weight               0.0010956  0.0001292   8.480  < 2e-16 ***
Guarantee            0.0418452  0.0092706   4.514 7.50e-06 ***
BOVAG                0.0460120  0.0158746   2.898  0.00387 ** 
KM_log               0.0053428  0.0047217   1.132  0.25822    
QuartTax:Fuel_clean  0.0027142  0.0003040   8.927  < 2e-16 ***
Age:KM_log      